In [ ]:
import gym
import random
import numpy as np
import pygad
import networkx as nx
shouldPickUp = False
shouldDropOff = True
hasDrop = False
penalty = 0
PickUped = False
yy = 0

def initial_enviroment():
    global PickUped
    PickUped = False
    shouldPickUp = True
    shouldDropOff = False
    env = gym.make("Taxi-v3", render_mode = 'ansi').env
    env.reset(seed = 20)
    def g1s():

        state = env.s
        taxi_row, taxi_col, pass_loc, dest_idx = env.decode(state)
        pickup_loc = env.locs[pass_loc]
        dropoff_loc = env.locs[dest_idx]
        
        return taxi_row, taxi_col, dropoff_loc, pickup_loc

    def g2s():

        state = env.s
        taxi_row, taxi_col, pass_loc, dest_idx = env.decode(state)
        dropoff_loc = env.locs[dest_idx]
        
        return taxi_row, taxi_col, dropoff_loc


    def p01():
        env.step(0)
        return "J01"

    def p011():
        env.step(1)
        return "J01"

    def P02():
        env.step(2)
        return "J01"

    def p03():
        env.step(3)
        return "J01"

    def p04():
        global PickUped
        env.step(4)
        PickUped = True
        return "J01"


    def J01():
        if shouldPickUp == True and shouldDropOff == False:
            return "J03"
        elif shouldPickUp == False and shouldDropOff == True:
            return "J02"


    def J03():
        taxi_row, taxi_col, dropoff_loc, pickup_loc = g1s()
        if pickup_loc[0] == taxi_row and pickup_loc[1] == taxi_col:  # Pickup location is same as taxi location
            return "p04"
        elif pickup_loc[1] > taxi_col:  # Move East
            return "J04" 
        elif pickup_loc[1] < taxi_col:  # Move West
            return "J05"
        elif pickup_loc[0] > taxi_row:  # Move South
            return "p01"
        elif pickup_loc[0] < taxi_row:  # Move North
            return "p011"

    #move east branch
    def J04():
        if shouldPickUp == True and shouldDropOff == False:
            taxi_row, taxi_col, dropoff_loc, pickup_loc = g1s()
        elif shouldPickUp == False and shouldDropOff == True:
            taxi_row, taxi_col, dropoff_loc= g2s()
        specific_locations = [(0, 1), (1, 1), (3, 0), (4, 0), (3, 2), (4, 2)]
        if (taxi_row, taxi_col) in specific_locations:
            # Check if increasing taxi_row and the new location is still in the specific locations
            if taxi_row < 2:
                return "p01"#move south
            else:
                return "p011"#move North
        return "P02"

    #move west branch
    def J05():
        if shouldPickUp == True and shouldDropOff == False:
            taxi_row, taxi_col, dropoff_loc, pickup_loc = g1s()
        elif shouldPickUp == False and shouldDropOff == True:
            taxi_row, taxi_col, dropoff_loc= g2s()
        specific_locations = [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]
        if (taxi_row, taxi_col) in specific_locations:
            # Check if increasing taxi_col and the new location is still in the specific locations
            if taxi_row < 2:
                return "p01"  #move south
            else:
                return "p011"  #move North
        return "p03"


    functionsLibrary1 = {
        "J01": J01,
        "J03": J03,
        "J04": J04,
        "J05": J05,
        "p01": p01,
        "p011": p011,
        "P02": P02,
        "p03": p03,
        "p04": p04
    }

    total = 0
    current = "J01"

    while total < 30:
        if PickUped == True:
            break
        current_function = functionsLibrary1[current]
        returned_value = current_function()
        total += 1
        current = returned_value


    return env



def renderMap(env):
    ok = env.render()
    print(ok)

def g1(env):

    state = env.s
    taxi_row, taxi_col, pass_idx, dest_idx = env.decode(state)
    dropoff_loc = env.locs[dest_idx]
    return taxi_row, taxi_col, dropoff_loc

def P10(env):
    env.step(0)
    return 2

# - 1: move north
def P11(env):
    env.step(1)
    return 2

# - 2: move east
def P12(env):
    env.step(2)
    return 2

# - 3: move west
def P13(env):
    env.step(3)
    return 2

def P5(env):
    global yy
    global hasDrop
    global penalty
    global shouldPickUp
    global shouldDropOff
    temp = env.step(5)
    reward = temp[1]
    if reward == 20:
        hasDrop = True
        shouldPickUp = False
        shouldDropOff = False
        yy = 7
        # print("hahahahahahahah")
    else:
        return 2
    return 2

def J2(env):
    
    taxi_row, taxi_col,dropoff_loc = g1(env)

    if dropoff_loc[0] == taxi_row and dropoff_loc[1] == taxi_col:
        return 11
    elif dropoff_loc[1] > taxi_col:
        return 4
    elif dropoff_loc[1] < taxi_col:
        return 5
    elif dropoff_loc[0] < taxi_row:
       return 7
    elif dropoff_loc[0] > taxi_row:
        return 6
    
def J14(env):

    taxi_row, taxi_col, _= g1(env)

    specific_locations = [(0, 1), (1, 1), (3, 0), (4, 0), (3, 2), (4, 2)]

    if (taxi_row, taxi_col) in specific_locations:
        # Check if increasing taxi_row and the new location is still in the specific locations
        if taxi_row < 2:
            return 6 #move south
        else:
            return 7 #move North
    return 8

def J15(env):

    taxi_row, taxi_col, _= g1(env)
    specific_locations = [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]
    if (taxi_row, taxi_col) in specific_locations:
        # Check if increasing taxi_col and the new location is still in the specific locations
        if taxi_row < 2:
            return 6  #move south
        else:
            return 7  #move North
    return 9

functions_library = {
    2: J2,
    4: J14,
    5: J15,
    6: P10,
    7: P11,
    8: P12,
    9: P13,
    11: P5
}


def get_infor_from_env(env):#this have porblem
    information = []
    information.append(2)
    information.append(11)
   
    taxi_row, taxi_col,dropoff_loc= g1(env)
    if dropoff_loc[0] == taxi_row and dropoff_loc[1] == taxi_col:
        #information.append(11)
        return [11,11]
    elif dropoff_loc[1] > taxi_col:
        information.append(4)
        specific_locations = [(0, 1), (1, 1), (3, 0), (4, 0), (3, 2), (4, 2)]
        if (taxi_row, taxi_col) in specific_locations:
                # Check if increasing taxi_row and the new location is still in the specific locations
            if taxi_row < 2:
                information.append(6)
            elif taxi_row > 2:
                information.append(7)
            else:
                information.append(8)
    elif dropoff_loc[1] < taxi_col:
        information.append(5)
        specific_locations = [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]
        if (taxi_row, taxi_col) in specific_locations:
                # Check if increasing taxi_col and the new location is still in the specific locations
            if taxi_row < 2:
                information.append(6)
            elif taxi_row > 2:
                information.append(7)
            else:
                information.append(9)
    elif dropoff_loc[0] < taxi_row:
        information.append(7)
    elif dropoff_loc[0] > taxi_row:
        information.append(6)
    information.sort()

    return information



def generate_2d_array(num_arrays, seed=None):
    if seed is not None:
        random.seed(seed)

    adj_list = {
        2: [6, 7, 4, 5, 11],
        4: [8, 7, 6],
        5: [6, 7, 9],
        6: [2],
        7: [2],
        8: [2],
        9: [2],
        11: [2]
    }

    long_1d_arrays = []

    for _ in range(num_arrays):
        long_1d_array = []

        for node, connections in adj_list.items():
            long_1d_array.append(node)

            num_connections = len(connections)
            if num_connections < 2:
                num_connections += 2
            long_1d_array.append(num_connections)

            connected_nodes = random.sample(sorted(set(range(2, 11)) - {node,3,10}), num_connections)
            long_1d_array.extend(connected_nodes)

        long_1d_arrays.append(long_1d_array)

    return long_1d_arrays

initial_population = generate_2d_array(20)

def find_node_index(node_id,graph_array):
    index = 0
    while index < len(graph_array):
        if graph_array[index] == node_id:
            return index
        num_connections = graph_array[index + 1]
        index += num_connections + 2
    return -1  # Return -1 if the node_id is not found in the grap

def go_thru(graph_array):
    ok = []
    env = initial_enviroment()

    index = 0
    step = 1

    while step <= 40:

        node_id = graph_array[index]

        ok.append(node_id)
        a = functions_library[node_id](env)
        con = graph_array[index+2:index+2+graph_array[index+1]]

        if a in con:
            index = find_node_index(a, graph_array)
        else:
            b_connections = get_infor_from_env(env)# get infor from enviroment 
            found_connection = None
            for connection in con:
                if connection in b_connections:
                    found_connection = connection
                    index = find_node_index(found_connection, graph_array)
                    break
            else:
                break
        step += 1
    
    #renderMap(env)
    return ok,env

def Astar(taxi_row, taxi_col, location):
    rows, cols = 5, 5
    G = nx.Graph()

    for y in range(rows):
        for x in range(cols):
            G.add_node((y, x))

    cannot_move_right = [(3, 0), (4, 0), (3, 2), (4, 2), (0, 1), (1, 1)]
    cannot_move_left = [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]

    for y in range(rows):
        for x in range(cols):
            if (y, x) not in cannot_move_right and x + 1 < cols:
                G.add_edge((y, x), (y, x + 1))
            if (y, x) not in cannot_move_left and x - 1 >= 0:
                G.add_edge((y, x), (y, x - 1))
            if y + 1 < rows:
                G.add_edge((y, x), (y + 1, x))
            if y - 1 >= 0:
                G.add_edge((y, x), (y - 1, x))

    start = (taxi_row, taxi_col)
    goal = location

    path = nx.astar_path(G, start, goal)

    distance = len(path) - 1

    return distance

def fitness_equation(hasDropoff,initial_drop_distance,final_drop_distance):

    droped = 0
    if hasDropoff:
        droped = 1

    closedistace = 0
    if final_drop_distance < initial_drop_distance:
        closedistace = 30*max((initial_drop_distance-final_drop_distance),0)
    fitness_score = 100*droped + closedistace
    penalty = 0
    return fitness_score

initial_population = generate_2d_array(20)

def fitness_func(ga_instance, solution, solution_idx):
    global hasDrop
    hasDrop = False
    env = initial_enviroment()

    state_values = g1(env)
    taxi_row, taxi_col, dropoff_loc = state_values
    initial_distance = Astar(taxi_row, taxi_col, dropoff_loc)

    ok, newEnv = go_thru(solution)

    state_values = g1(newEnv)

    taxi_row, taxi_col, dropoff_loc = state_values
    final_distance = Astar(taxi_row, taxi_col, dropoff_loc)

    fs = fitness_equation(hasDrop,initial_distance,final_distance)

    return fs

def random_pick_2d_arrays(arr_2d, n):
    if len(arr_2d) < 2:
        raise ValueError("The input 2D array must contain at least 2 arrays.")

    picked_arrays = []
    for _ in range(n):
        indices = random.sample(range(len(arr_2d)), 2)
        selected_arrays = [arr_2d[i] for i in indices]
        picked_arrays.append(selected_arrays)

    return picked_arrays

def mutation(a):
    i = 0
    new_a = []

    while i < len(a):
        nodeID = a[i]
        num_connections = a[i + 1]
        a_connections = a[i + 2:i + 2 + num_connections]

        modified_connections = [random.choice([current_connection, random.randint(2, 11)]) if random.random() <= 0.001 else current_connection for current_connection in a_connections]

        new_a.append(nodeID)
        new_a.append(num_connections)
        new_a.extend(modified_connections)

        i += 2 + num_connections

    return new_a

def mutation_func(offspring, ga_instance):
  
    result = []
    for indi in offspring:
        a = mutation(indi)
        result.append(np.array(a))


    re = np.array(result)
    return re

def crossover(a, b):
    i = 0
    new_a = []
    new_b = []
    swap_rate=0.2
    while i < len(a):
        num_connections = a[i + 1]
        a_connections = a[i + 2:i + 2 + num_connections]
        b_connections = b[i + 2:i + 2 + num_connections]

        # Iterate through the connections and swap with a 0.6 probability for each digit
        for j in range(num_connections):
            dice_roll = random.random()  # Generate a random number between 0 and 1
            if dice_roll <= swap_rate:
                # Swap the digits
                a_connections[j], b_connections[j] = b_connections[j], a_connections[j]

        # Append swapped connections to the new arrays
        new_a.extend([a[i], num_connections])
        new_b.extend([b[i], num_connections])
        new_a.extend(a_connections)
        new_b.extend(b_connections)

        i += 2 + num_connections

    return new_a, new_b

def crossover_func(parents, offspring_size, ga_instance):
    pickTimes = int((offspring_size[0] + 1)/2)
    picker = random_pick_2d_arrays(parents,pickTimes)
    result = []
    for pair in picker:
         new_a, new_b = crossover(pair[0], pair[1])
         result.append(np.array(new_a))  # Ensure the offspring are NumPy arrays
         result.append(np.array(new_b))
    
   
    result.pop(int(offspring_size[0]))
    re = np.array(result)
    return re

ga_instance = pygad.GA(num_generations=2000,
                       gene_type=int,
                       initial_population=initial_population,
                       num_parents_mating=10,
                       fitness_func=fitness_func,
                       parent_selection_type="tournament",
                       K_tournament=4,
                       crossover_type= crossover_func,
                       mutation_type=mutation_func)

ga_instance.run()
fitness = ga_instance.cal_pop_fitness()

print(yy)
print(fitness[0])
